<center>
    <h1>TUGAS III - Individu</h1>
    <h2>DATA WAREHOUSE</h2>
    <h3>Maftuh Mashuri (11160940000076)</h3>
</center>

# Fungsi untuk koneksi database

In [36]:
def connect(nama_db = "kantor", password = "maftuh2003"):
    import psycopg2
    conn = psycopg2.connect(database = nama_db, user = "postgres", password = password, host = "localhost", port = "5432")
    return conn

# Fungsi untuk CRUD
<p>Fungsi ini untuk melakukan running query ke database dengan input <i>query</i> yaitu string berisi query perintah untuk database dan <i>select</i> dengan type data boolean karena hanya perintah SELECT yang mengeluarkan output</p>

In [37]:
def execute(query, select = True):
    conn = connect()
    cur = conn.cursor()
    cur.execute(query)
    if select:
        return cur.fetchall()
    else:
        conn.commit()
    conn.close()

In [51]:
# query untuk mengosongkan tabel
query = "DELETE FROM karyawan; DELETE FROM cuti_karyawan;"
execute(query, False)

In [39]:
# query untuk mengosongkan tabel
query = "DROP TABLE karyawan; DROP TABLE cuti_karyawan;"
execute(query, False)

# b) Membuat tabel-tabel pada database kantor.

In [40]:
query = '''
    CREATE TABLE IF NOT EXISTS karyawan(
    Nomor_induk VARCHAR(10) NOT NULL,
    Nama VARCHAR(30),
    Alamat TEXT,
    Tanggal_lahir DATE,
    Tanggal_masuk DATE,
    PRIMARY KEY(Nomor_induk)
    );

    CREATE TABLE IF NOT EXISTS cuti_karyawan(
    id INT NOT NULL,
    Nomor_induk VARCHAR(10),
    Tanggal_mulai DATE,
    Lama_cuti SMALLINT,
    Keterangan TEXT,
    PRIMARY KEY(id));
    '''
execute(query, False)
print("Membuat tabel berhasil")

Membuat tabel berhasil


# c) Mengisi tabel-tabel

In [41]:
query = '''
    INSERT INTO karyawan (Nomor_induk, Nama, Alamat, Tanggal_lahir, Tanggal_masuk)
    VALUES 
    ('IP06001', 'Agus', 'Jln. Gajah Mada 115A, Jakarta Pusat', '1970-08-01', '2006-07-07'),
    ('IP06002', 'Amin', 'Jln. Bungur sari v No. 178 Bandung', '1977-05-03', '2006-07-07'),
    ('IP06003', 'Yusuf', 'Jln. Yosodpuro 15, Surabaya', '1973-08-09', '2006-07-07'),
    ('IP07004', 'Alyssa', 'Jln. Cendana No. 6 Bandung', '1983-02-14', '2007-01-05'),
    ('IP07005', 'Maulana', 'Jln. Ampera Raya No 1', '1985-10-10', '2007-02-05'),
    ('IP07006', 'Afika', 'Jln. Pejaten Barat No 6A', '1987-03-09', '2007-06-09'),
    ('IP07007', 'James', 'Jln. Padjadjaran No. 111, Bandung', '1988-05-19', '2007-06-09'),
    ('IP09008', 'Octavanus', 'Jln. Gajah Mada 101, Semarang', '1988-10-07', '2008-08-08'),
    ('IP09009', 'Nugroho', 'Jln. Duren Tiga 196, Jakarta Selatan', '1988-01-20', '2008-11-11'),
    ('IP090010', 'Raisa', 'Jln. Nangka Jakarta Selatan', '1989-12-29', '2009-02-09');
    
    INSERT INTO cuti_karyawan(id, Nomor_induk, Tanggal_mulai, Lama_cuti, Keterangan)
    VALUES
    (1, 'IP06001', '2012-02-01', 3, 'Acara keluar'),
    (2, 'IP06001', '2012-02-13', 2, 'Anak sakit'),
    (3, 'IP07007', '2012-02-15', 1, 'Nenek sakit'),
    (4, 'IP06003', '2012-02-17', 1, 'Mendaftar sekolah anak'),
    (5, 'IP07006', '2012-02-20', 5, 'Menikah'),
    (6, 'IP07004', '2012-02-27', 1, 'Imunisasi anak');
    '''
execute(query, False)
print("Input data berhasil")

Input data berhasil


# d) Tentukan empat orang karyawan yang pertama kali masuk (bergabung)

In [42]:
query = "SELECT Nama FROM karyawan ORDER BY Tanggal_masuk LIMIT 4;"
data = execute(query)

print("Empat orang karyawan yang pertama kali masuk adalah : ")
for user in data:
    print("-", user[0])

Empat orang karyawan yang pertama kali masuk adalah : 
- Agus
- Yusuf
- Amin
- Alyssa


# e) Tentukan dua orang karyawan yang terakhir kali masuk (bergabung)

In [43]:
query = "SELECT Nama FROM karyawan ORDER BY Tanggal_masuk DESC LIMIT 2;"
data = execute(query)

print("Dua orang karyawan yang terakhir kali masuk adalah : ")
for user in data:
    print("-", user[0])

Dua orang karyawan yang terakhir kali masuk adalah : 
- Raisa
- Nugroho


# f) Tentukan nama karyawan yang paling banyak mengambil cuti beserta keterangan cutinya.

In [44]:
query = '''
    SELECT 
        karyawan.Nama, cuti_karyawan.keterangan
    FROM 
        karyawan
    JOIN 
        cuti_karyawan 
    ON 
        cuti_karyawan.Nomor_induk = karyawan.Nomor_induk
    JOIN 
        (SELECT Nomor_induk, COUNT(Nomor_induk) AS total_cuti FROM cuti_karyawan GROUP BY Nomor_induk) grup_cuti
    ON 
        grup_cuti.Nomor_induk = karyawan.Nomor_induk
    WHERE 
        grup_cuti.total_cuti = 
            (SELECT MAX(x.max) FROM (SELECT COUNT(Nomor_induk) AS max FROM cuti_karyawan GROUP BY Nomor_induk) x);
    '''
data = execute(query)

print("Karyawan yang paling banyak mengambil cuti adalah:")
for i in range(len(data)):
    if i < len(data)-1:
        if data[i][0] == data[i+1][0]:
            print("-> " + data[i][0] + ", dengan keterangan:")
    print(" -", data[i][1])
    
################ Tambahan karyawan yang paling lama mengambil cuti ############################
query = '''
    SELECT karyawan.Nama, cuti_karyawan.keterangan 
    FROM karyawan 
    JOIN cuti_karyawan ON karyawan.Nomor_induk = cuti_karyawan.Nomor_induk 
    WHERE cuti_karyawan.lama_cuti = (SELECT MAX(Lama_cuti) FROM cuti_karyawan)
    '''

print()
print("Karyawan yang paling lama mengambil cuti adalah:")
data = execute(query)
for user in data:
    print(user[0], "dengan keterangan", user[1])

Karyawan yang paling banyak mengambil cuti adalah:
-> Agus, dengan keterangan:
 - Acara keluar
 - Anak sakit

Karyawan yang paling lama mengambil cuti adalah:
Afika dengan keterangan Menikah


# g) Tentukan nama karayawan yang memiliki umur paling tua

In [45]:
query = "SELECT Nama FROM karyawan WHERE Tanggal_lahir = (SELECT MIN(Tanggal_lahir) FROM karyawan);"
data = execute(query)

print("Karyawan yang memiliki umur paling tua adalah:")
for user in data:
    print("-", user[0])

Karyawan yang memiliki umur paling tua adalah:
- Agus


# h) Tentukan nama karyawan yang memiliki umur paling muda

In [46]:
query = "SELECT Nama FROM karyawan WHERE Tanggal_lahir = (SELECT MAX(Tanggal_lahir) FROM karyawan);"
data = execute(query)

print("Karyawan yang memiliki umur paling muda adalah:")
for user in data:
    print("-", user[0])

Karyawan yang memiliki umur paling muda adalah:
- Raisa


# i) Tentukan nama-nama karyawan yang paling dulu masuk kerja setelah cuti

In [47]:
query = '''
    SELECT Nama
    FROM karyawan
    JOIN cuti_karyawan ON cuti_karyawan.Nomor_induk = karyawan.Nomor_induk
    WHERE cuti_karyawan.Lama_cuti = (SELECT MIN(Lama_cuti) FROM cuti_karyawan);
    '''
data = execute(query)

print("Karyawan yang paling dulu masuk kerja setelah cuti adalah:")
for user in data:
    print("-", user[0])

Karyawan yang paling dulu masuk kerja setelah cuti adalah:
- Yusuf
- Alyssa
- James


# j) Tentukan total cuti yang diambil para karyawan.

In [48]:
query = '''
    SELECT karyawan.Nama, COUNT(cuti_karyawan.Nomor_induk)
    FROM karyawan
    JOIN cuti_karyawan ON cuti_karyawan.Nomor_induk = karyawan.Nomor_induk
    GROUP BY karyawan.Nomor_induk;
    '''
data = execute(query)
print("Total cuti yang diambil para karyawan adalah:")
for user in data:
    print("-", user[0], "sebanyak", user[1], "kali")
print("Sisa karyawan lainnya tidak mengambil cuti.")

############# Tambahan Total lama cuti semua karyawan ##########
query = '''
    SELECT SUM(Lama_cuti) AS jumlah
    FROM cuti_karyawan;
    '''
data = execute(query)
print("\nTotal lama cuti yang diambil para karyawan sebanyak", data[0][0], "hari")

Total cuti yang diambil para karyawan adalah:
- Yusuf sebanyak 1 kali
- Afika sebanyak 1 kali
- Alyssa sebanyak 1 kali
- Agus sebanyak 2 kali
- James sebanyak 1 kali
Sisa karyawan lainnya tidak mengambil cuti.

Total lama cuti yang diambil para karyawan sebanyak 13 hari


# k) Apakah ada karyawan yang tidak mengambil cuti? Jika iya tentukan nama-nama karyawan tersebut.
<p>Ya. Sesuai dengan output sebelumnya, ada karyawan yang tidak menggambil cuti.</p>

In [49]:
query = '''
    SELECT Nama
    FROM karyawan
    WHERE Nomor_induk NOT IN (SELECT Nomor_induk FROM cuti_karyawan);
    '''
data = execute(query)
print("Karyawan yang tidak mengambil cuti adalah:")
for user in data:
    print("-", user[0])

Karyawan yang tidak mengambil cuti adalah:
- Amin
- Maulana
- Octavanus
- Nugroho
- Raisa
